In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plotwidth=40

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp

import seaborn as sns
import matplotlib.pyplot as plt

from natsort import natsort_keygen

import statsmodels.api as sm
import scipy as sp

import os
import json
import datetime
import psycopg2
import netrc
import re
import yaml
from tqdm.notebook import tqdm, trange
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

# Globals

In [ ]:
datadir = '.'
plotdir = 'plots'
vpipe_working = 'working' # V-pipe's working directory

# Input
tally_mut = os.path.join(datadir, 'tallymut_line.tsv')
cooc_table = {
    'v3':os.path.join(vpipe_working, 'ww-cooc.v3.csv'),
    'v4':os.path.join(vpipe_working, 'ww-cooc.v4.csv'),
}
amplicons_yaml = {
    'v3':os.path.join(vpipe_working, 'amplicons.v3.yaml'),
    'v4':os.path.join(vpipe_working, 'amplicons.v4.yaml'),
}

# Select
start_date = '2020-12-08'
start_date_klzh = '2021-08-15'
todaydate = datetime.date.today().strftime("%Y-%m-%d")
cities_list=['Altenrhein (SG)', 'Chur (GR)', 'Genève (GE)', 'Laupen (BE)',
       'Lausanne (VD)', 'Lugano (TI)', 'Zürich (ZH)', 'Kanton Zürich', 'Basel (catchment area ARA Basel)']
# variants that will be displayed here AND uploaded:
variants_list_upload=['al', 'be', 'ga', 'de', 'ka', 'om1','om2',] # C36, AY42, mu, B16173, d614g
# extra variants that will be ONLY display here, but NOT uploaded (typically for variant not present yet)
variants_list=variants_list_upload+['BA1','BA2',]
variants_pangolin={'al':'B.1.1.7','be':'B.1.351','ga':'P.1','C36':'C.36.3','ka':'B.1.617.1','de':'B.1.617.2','AY42':'AY.4.2','B16173':'B.1.617.3','om1':'BA.1','om2':'BA.2','BA1':'custom-BA.1','BA2':'custom-BA.2'}
exclusive_list=['be','ga'] # list of variants where we should apply filtering
exclude_from=['al','be','ga' ] #,'IN2'] #,'C36','IN1','IN2','IN3'] # filter against these variants
# currently, heatmaps can only display amplicons from 1 single protocol
amplicons_proto = { v:'v4' for v in ['om1','om2','BA1','BA2'] }
amplicons_proto.update({ v:'v3' for v in variants_list if v not in amplicons_proto.keys() })
display(amplicons_proto)

# Output
update_data_file = os.path.join(datadir, 'ww_update_data_heatmap.json')

# Load Data

In [ ]:
amplicons = {}
# process amplicons definition for each proto
for (pro,fname) in amplicons_yaml.items():
    with open(fname, 'rt') as yf:
        amp_str = yaml.safe_load(yf)
    # process each enty
    for (aname,adata) in amp_str.items():
        # parse: 88_BA1_BA2
        pn=aname.split("_")
        num=int(pn[0]) # number
        for av in pn[1:]: # variants
            # only process if we're supposed to use that version for that variant:
            if amplicons_proto.get(av, None) != pro:
                continue
            # only process non-indels (because indels aren't on heatmap)
            muts = [ f"{p3}{m3}" for p3,m3 in sorted([(p2,m2) for p,m in adata[4].items() for p2,m2 in zip(range(p,p+len(m)),m) ]) ] #if m3[0] not in "-+" ]
            if len(muts)==0:
                continue
            # add to dict
            add = { num: muts}
            print(f"{pro} {av} {num} {','.join(muts)}/{len(muts)}")
            if av not in amplicons:
                amplicons[av] = add
            else:
                amplicons[av].update(add)
amplicons={ v:dict(sorted(amps.items()))  for v,amps in amplicons.items() }
display(amplicons)
#    'IN1': { 76: ['22917G', '23012C'], },
#    'IN3': { 76: ['22917G', '23012C'], },
#    'IN2': { 76: ['22917G', '22995A'],
#             91: ['27638C', '27752T'], },
#    'BR':  { 71: ['21621A', '21638T'],  # ,'21614T'], # common mutations are removed from the plot
#             # 95: ['28877T',  '28878C'], # not part of the signature mutations
#           },
#    'AY42':{ 73: ['21995C','22227T'], },
#
#    'om':  { 75: ['22578A','22673C','22674T','22679C','22813T'],
# v4 
#             75_om[22578A,22673CTC,22679C]
#             76: ['22882G','22898A','22992A','23013C','23040G','23048A','23055G'],
#             78: ['23525T','23599G'],
#             88: ['26577G','26709A'],
#           },

## Load WWTP Sequencing Data

#### Tally of mutations in samples

This loads the TSV file generated by the [mut-table.ipynb](mut-table.ipynb) notebook.

In [ ]:
df = pd.read_csv(tally_mut, sep='\t', parse_dates=['date'])

df['mutation'] = df['pos'].astype(str) + df['base']
df#.head()
sorted(df['date'].unique())
df[df['date'] == '2021-12-27']
sorted(df[df['plantname'] == 'Basel (catchment area ARA Basel)']['date'].unique())
df[df['plantname'] == 'Basel (catchment area ARA Basel)']['batch'].unique()
df[df['batch'] == '20220110_HKLFGDRXY']['plantname'].unique()

In [ ]:
# remove problematic mutations (from mutation influence diagnostic) (hard-coded now)

df = df[~(df['mutation'].isin(["28461G", "11201G", "26801C"]))]

### Cooccurrences by cojac

This loads the CSV generated by the `ww.bsub` LSF job.

In [ ]:
df_cooc_raw = { pro:pd.read_csv(fname, sep=',') for (pro,fname) in cooc_table.items() }
#, index_col=['sample','batch'])

#df['mutation'] = df['pos'].astype(str) + df['base']

df_cooc_raw['v3'].head()

### Handle duplicates

The code in the notebook will have trouble if some indexes are duplicated

In [ ]:
# Look for duplicated samples with suffixes
df[(~df['plantname'].isna()) & (df['date'] >= start_date) & (df['base'] != '-') & df.duplicated(subset=['plantname','date','batch','mutation'], keep=False)]#.groupby('batch').count()

In [ ]:
# HACK workaround for a duplicated sample
df['batch'].loc[df['sample']=='A1_05_2021_05_19_CATTCGGA-TTTCCATC']='20210604_JN8TR_CATTCGGA-TTTCCATC'
for pro in df_cooc_raw.keys():
    df_cooc_raw[pro]['batch'].loc[df_cooc_raw[pro]['sample']=='A1_05_2021_05_19_CATTCGGA-TTTCCATC']='20210604_JN8TR_CATTCGGA-TTTCCATC'

In [ ]:
# HACK workaround for a duplicated sample
df['batch'].loc[df.duplicated(subset=['plantname','date','batch','mutation'], keep=False) & df['sample'].str.contains('_P$')]='Promega'
for pro in df_cooc_raw.keys():
    df_cooc_raw[pro]['batch'].loc[df_cooc_raw[pro]['sample'].str.contains('_P$')]='Promega'

In [ ]:
# Look for duplicated samples with suffixes
df[(~df['plantname'].isna()) & (df['date'] >= start_date) & (df['base'] != '-') & df.duplicated(subset=['plantname','date','batch','mutation'], keep=False)].groupby('batch').count()

In [ ]:
df_cooc_raw['v3']

In [ ]:
# plantcode and date from mut_table to cooc

df_map=df[['sample','batch','plantname','date']].drop_duplicates(ignore_index=True).set_index(['sample','batch'])
df_cooc={ pro:df_cooc_raw[pro].merge(df_map[['plantname','date']], how='left', on=['sample','batch'], copy=False, validate='many_to_one').set_index(['plantname','date','batch']) for pro in df_cooc_raw.keys() }
df_cooc['v3']#.head()

In [ ]:
for (pro,pro_df) in df_cooc.items():
    pro_df.to_csv(f"df_cooc.{pro}.csv")

In [ ]:
def mutfilter(var, exclusive=False):
    # if exclusive is set on true, it will filter only those mutation which are variant specific and DO NOT show up in other variants
    # e.g.: used to exclude 23063T (V501Y) as all variant have it
    return (df[exclude_from].fillna(0) == ['mut' if v==var else 0 for v in exclude_from]).all(axis=1) if exclusive else (df[var] == "mut")
mutfilter('al', exclusive=True)

In [ ]:
df_wide = {}
amp_col = {}
rx_num = re.compile(r"^(\d+)")
for var in tqdm(variants_list):
    df_wide[var] = (
        # for the remaining mutations
        df[(~df['plantname'].isna()) & (df['date'] >= start_date) & (df['base'] != '-') & (mutfilter(var, exclusive=(var in exclusive_list)))]
         .pivot(index=['plantname', 'date', 'batch'], columns=['mutation'], values='frac')
         .sort_index(axis=1, key=natsort_keygen())
    )
    # add amplicons
    if var in amplicons and var in amplicons_proto:
        amp_col[var] = []
        for amp,muts in amplicons[var].items():
            # transfer this column from the coocurence table...
            aname =  f"Amp {amp}"
            amplicon_table = df_cooc[amplicons_proto[var]].loc[(df_cooc[amplicons_proto[var]]['amplicon'] == amp) & (df_cooc[amplicons_proto[var]][var]==1), ['frac']]
            amp_col[var] += [ aname ]
            # (filled with NA on dates for which we don't have coocurence, and subset to target indices)
            missing_cooc = df_wide[var].index.difference(amplicon_table.index)
            na_filled_col = pd.concat([amplicon_table, pd.DataFrame(index=missing_cooc, columns=amplicon_table.columns).fillna(np.nan)])
            # ...into that position
            if muts[-1] in df_wide[var].columns:
                cooccol=1+df_wide[var].columns.get_loc(muts[-1])
            else:
                # [inefficient] roughly the same idea as get_loc(method='fill'), but our index isn't monotonic
                cooccol=0
                srch=int(rx_num.search(muts[-1]).group()) # to number
                for idx in df_wide[var].columns:
                    cooccol+=1
                    inum=rx_num.search(idx) # begins with numbe?
                    if inum is not None and int(inum.group()) > srch: #numerical comparison
                        break
            df_wide[var].insert(loc=cooccol, column=aname, value=na_filled_col.loc[df_wide[var].index], allow_duplicates=False)

df_wide['ga'].head()

In [ ]:
df_wide['om1'].loc['Basel (catchment area ARA Basel)']

In [ ]:
df_wide_cov = (
    df[(~df['plantname'].isna()) & (df['date'] >= '2020-12-08') & (df['base'] != '-') & (df['al'] == "mut")]
    .pivot(index=['plantname', 'date', 'batch'], columns=['mutation'], values='cov')
    .sort_index(axis=1, key=natsort_keygen())
)

df_wide_counts = (
    df[(~df['plantname'].isna()) & (df['date'] >= '2020-12-08') & (df['base'] != '-') & (df['al'] == "mut")]
    .pivot(index=['plantname', 'date', 'batch'], columns=['mutation'], values='var')
    .sort_index(axis=1, key=natsort_keygen())
)

# Heatmaps

In [ ]:
for city in tqdm(cities_list, desc='Cities', position=0): # ['Basel (catchment area ARA Basel)']:#
    for var in tqdm(variants_list, desc='Variants', position=1):
        tmp=df_wide[var].loc[city]

        # drawing box decorations:
        #
        #      22917G╮        27972T╮
        #      22995A┤        28048T┤
        # Amplicon 76╯        28111G┪
        #                Amplicon 92┩
        #      22679C┓        28280C┤
        #      22688G┨        28281T┤
        #      22775A┨        28282A┤
        # Amplicon 75┩   Amplicon 93╯
        #      22786C┤
        # Amplicon 76╯   Amplicon 73-

        if var in amplicons:
            rmap={}
            endbox={}
            l_amp = 'x'

            # mutations
            for amp,muts in amplicons[var].items():
                box = '╮' #'┐'
                dblbox = '┪'
                for m in muts:
                    if m in tmp.columns:
                        if m not in rmap:
                            rmap[m]=f"{m}{box}"
                        else:
                            rmap[m]=f"{m}{'┓' if rmap[m][-1] == '╮' else dblbox}"
                            dblbox = '┨'
                            endbox[l_amp] = "┩"
                        box = '┤'
                        endbox[f"Amp {amp}"] = '╯' # "┘"
                l_amp=f"Amp {amp}"
            #display(endbox)

            # amplicons
            for a in amp_col[var]:
                rmap[a]=f"{a}{endbox.get(a,'-')}"
            #print(rmap)
            tmp.rename(columns=rmap,inplace=True)

        plt.figure(figsize=(plotwidth,7)) # 17
        trns=tmp.T
        sns.heatmap(
            data=trns.fillna(np.nan), #applymap(lambda x: np.nan if pd.isna(x) else x),
            annot=trns,
            fmt='.1g',
            square=False, cbar=False,
            cmap=sns.color_palette("viridis", as_cmap=True)
        ).set_title(f"{city} - {variants_pangolin[var]}")
        plt.show()

# Prepare Heatmap Data for covSPECTRUM

In [ ]:
import warnings

with warnings.catch_warnings(record=True) as w:
    r_df={}
    for city in tqdm(cities_list, desc='Cities', position=0):
        r_df[city]={}
        for var in tqdm(variants_list, desc='Variants', position=1, leave=False):
            r_df[city][var] = df_wide[var].loc[city].dropna(axis=1, how='all').T  # .loc[28111:28111]

m_df={}
# import matplotlib.gridspec as gridspec
for city in tqdm(cities_list, desc='Cities', position=0):
    m_df[city]={}
    for var in tqdm(variants_list_upload, desc='Variants', position=1, leave=False):
        m_df[city][var] = r_df[city][var].T.groupby("date").agg("mean").asfreq('D').T

In [ ]:
m_df.keys()

In [ ]:
m_df['Altenrhein (SG)'].keys()

In [ ]:
m_df['Altenrhein (SG)']['al']

# Make data for covSPECTRUM

In [ ]:
update_data={ }
tdf={city:{}  for city in cities_list}
tdf_mat={city:{}  for city in cities_list}

# add 'undetermined' variant
update_data['undetermined'] = {
    # undertermined don't have their own proper heatmap (they are definted by signature missing for all other variant)
    city: {"mutationOccurrences": None} for city in cities_list
}

# process heatmaps normally for all other variants.
for var in tqdm(variants_list_upload, desc='Variants', position=0):
    update_data[variants_pangolin[var]] = { }
    for city in tqdm(cities_list, desc='Cities', position=1, leave=False): #['Basel (catchment area ARA Basel)']:#

#         tdf[city][var]["date"] = tdf[city][var]["date"].astype("str")

        tdf_mat[city][var] = m_df[city][var].T.melt(ignore_index=False, var_name="nucMutation", value_name="proportion").reset_index()
        tdf_mat[city][var]["date"] = tdf_mat[city][var]["date"].astype("str")
        #print(tdf_mat[city][var]["nucMutation"])
        if var in amplicons:
            ## drawing box decorations:
            ## ┌22917G
            ## ├22995A
            ## └Amplicon 76
            ## mutations
            #for amp,muts in amplicons[var].items():
            #    box = '╭' #'┌'
            #    endbox = '-'
            #    for m in muts:
            #        tdf_mat[city][var].loc[tdf_mat[city][var]["nucMutation"]==m,"nucMutation"]=f"{box}{m}"
            #        box = '├'
            #        endbox = '╰'#'└'
            ## amplicons
            #for a in amp_col[var]:
            #        tdf_mat[city][var].loc[tdf_mat[city][var]["nucMutation"]==a,"nucMutation"]=f"{endbox}{a}"

            # drawing box decorations:
            #
            # ╭22917G        ╭27972T
            # ├22995A        ├28048T
            # ╰Amplicon 76   ┢28111G
            #                ┡Amplicon 92
            # ┏22679C        ├28280C
            # ┠22688G        ├28281T
            # ┠22775A        ├28282A
            # ┡Amplicon 75   ╰Amplicon 93
            # ├22786C
            # ╰Amplicon 76  ~Amplicon 73~

            rmap={}
            endbox={}
            l_amp='x'
            # mutations
            for amp,muts in amplicons[var].items():
                box = '╭' #'┌'
                dblbox= '┢'
                for m in muts:
                    if m in m_df[city][var].index: #tdf_mat[city][var]["nucMutation"]:
                        if m not in rmap:
                            rmap[m]=f"{box}{m}"
                        else:
                            rmap[m]=f"{'┏' if rmap[m][0] == '╭' else dblbox}{m}"
                            dblbox='┠'
                            endbox[l_amp] = "┡"
                        box = '├'
                        endbox[f"Amp {amp}"] = '╰'#'└'
                l_amp=f"Amp {amp}"
            #print(endbox)

            # amplicons
            for a in amp_col[var]:
                if a in endbox:
                    rmap[a]=f"{endbox[a]}{a}"
                else:
                    # remove
                    tdf_mat[city][var].drop(tdf_mat[city][var].loc[tdf_mat[city][var]["nucMutation"]==a].index)
            
            # wite modifications
            #print(rmap)
            for orig,art in rmap.items():
                tdf_mat[city][var].loc[tdf_mat[city][var]["nucMutation"]==orig,"nucMutation"]=art
                  
          
        update_data[variants_pangolin[var]][city] = {
            #"updateDate": todaydate,
#             "timeseriesSummary": [dict(tdf[city][var].iloc[i,]) for i in range(tdf[city][var].shape[0])],
            "mutationOccurrences": [dict(tdf_mat[city][var].iloc[i,]) for i in range(tdf_mat[city][var].shape[0])]
        }

import json
with open(update_data_file, 'w') as file:
     file.write(json.dumps(update_data))

In [ ]:
# tdf_mat['Altenrhein (SG)'].keys()

In [ ]:
update_data['undetermined']

In [ ]:
update_data[variants_pangolin['om']]

In [ ]:
(0.5+len(df_wide['om'].loc['Zürich (ZH)'].columns)/5.1)

In [ ]:
(0.5+len(df_wide['ga'].loc['Zürich (ZH)'].columns)/5.1)